In [4]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
from torchtext.data import Field, LabelField, BucketIterator, TabularDataset
import spacy
import re

In [4]:
import drive.MyDrive.NLP_stuff.Sentiment_analysis.dataset_loader_torchtext as dlt
dataset_loader = dlt.CustomDatasetLoader('/content/drive/MyDrive/NLP_stuff/Sentiment_analysis/train.csv')

In [5]:
train_data, test_data = dataset_loader.get_data()
TEXT, LABEL = dataset_loader.get_fields()

In [6]:
print(len(train_data))

83005


In [251]:
import torch
from torchtext import data

BATCH_SIZE = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = dataset_loader.get_iterators(batch_size=BATCH_SIZE)

In [252]:
import torch.nn as nn

class RNN_model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * (1 + bidirectional), output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):        
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        return self.fc(hidden)

In [305]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 300
OUTPUT_DIM = 3
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN_model(INPUT_DIM, 
                  EMBEDDING_DIM, 
                  HIDDEN_DIM, 
                  OUTPUT_DIM, 
                  N_LAYERS, 
                  BIDIRECTIONAL, 
                  DROPOUT, 
                  PAD_IDX)

In [306]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,127,723 trainable parameters


In [307]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([11935, 100])


In [308]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.9377, -0.9760, -0.2120,  ...,  0.3367, -0.0054, -0.0457],
        [-0.0182, -0.1919,  0.2317,  ...,  0.9061, -1.6241, -0.5352],
        [ 0.4977, -0.7934,  0.7085,  ...,  0.0189, -1.2461,  1.6934],
        ...,
        [ 0.4558,  0.5267,  0.8105,  ...,  0.1609, -0.2292,  0.4388],
        [ 1.0380,  0.8317,  0.0391,  ..., -0.4941,  0.5906, -0.6920],
        [ 1.0988, -0.1175,  1.4507,  ...,  0.0638, -1.0160,  0.9116]])

In [309]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.4977, -0.7934,  0.7085,  ...,  0.0189, -1.2461,  1.6934],
        ...,
        [ 0.4558,  0.5267,  0.8105,  ...,  0.1609, -0.2292,  0.4388],
        [ 1.0380,  0.8317,  0.0391,  ..., -0.4941,  0.5906, -0.6920],
        [ 1.0988, -0.1175,  1.4507,  ...,  0.0638, -1.0160,  0.9116]])


In [310]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimizer = optim.Adam(model.parameters())
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1, end_factor=0.001, total_iters=6)
criterion = nn.CrossEntropyLoss().to(device)

model = model.to(device)

In [311]:
def categorical_accuracy(preds, y):
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [312]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in tqdm(iterator):
        
        optimizer.zero_grad()
        
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    scheduler.step()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [313]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [314]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [315]:
from tqdm import tqdm

In [316]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss <= best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print('\n')

100%|██████████| 2594/2594 [00:20<00:00, 128.26it/s]


Epoch: 01 | Epoch Time: 0m 20s
	Train Loss: 0.841 | Train Acc: 62.88%
	 Val. Loss: 0.732 |  Val. Acc: 69.05%




100%|██████████| 2594/2594 [00:20<00:00, 129.11it/s]


Epoch: 02 | Epoch Time: 0m 20s
	Train Loss: 0.732 | Train Acc: 68.86%
	 Val. Loss: 0.692 |  Val. Acc: 70.59%




100%|██████████| 2594/2594 [00:19<00:00, 131.81it/s]


...

Epoch: 08 | Epoch Time: 0m 19s
	Train Loss: 0.600 | Train Acc: 74.85%
	 Val. Loss: 0.661 |  Val. Acc: 72.29%




100%|██████████| 2594/2594 [00:19<00:00, 133.10it/s]


Epoch: 09 | Epoch Time: 0m 20s
	Train Loss: 0.600 | Train Acc: 74.96%
	 Val. Loss: 0.661 |  Val. Acc: 72.26%




 52%|█████▏    | 1345/2594 [00:09<00:08, 138.80it/s]


KeyboardInterrupt: ignored

In [277]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.658 | Test Acc: 72.30%


In [278]:
def predict_class(model, sentence):
    model.eval()
    tokenized = dlt.tokenize_english(sentence)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    preds = model(tensor, length_tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [279]:
sentence="Penalty on sleeping in RM is removed on behalf of Shubham Kumar"
pred_class = predict_class(model, sentence)
print(LABEL.vocab.itos[pred_class])

-1


In [280]:
import csv
import chardet

FILENAME = '/content/drive/MyDrive/NLP_stuff/Sentiment_analysis/test.csv'
with open(FILENAME, 'rb') as f:
    result = chardet.detect(f.read())

with open(FILENAME, newline='', encoding=result['encoding']) as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',', quotechar='"')

    # Skip the header row
    next(csv_reader)

    f_open = open("answer1.txt","w")
    for row in csv_reader:
        pred = LABEL.vocab.itos[predict_class(model, str(row[1]))]
        s= str(pred)
        s+='\n'
        f_open.write(s)
    f_open.close()